In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import altair as alt
import numpy as np
from st_aggrid import GridOptionsBuilder, AgGrid, GridUpdateMode, DataReturnMode

import openpyxl
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from itertools import islice


In [2]:

# open excel file
excel_file = "tenancy_list_07SEP2022.xlsx"
wb = openpyxl.load_workbook(excel_file, data_only=True)

# getting all sheets
sheets = wb.sheetnames

# getting a particular sheet
sheet_params = wb["params"]
sheet_data = wb["tenant"]
sheet_rental = wb["rental"]
sheet_yearly_rental = wb["yearly_rental"]
sheet_sc = wb["sc"]
sheet_yearly_sc = wb["yearly_sc"]
sheet_total = wb["total_rev"]
sheet_occ = wb["occ_rate"]

# reading parameters
date_start = sheet_params["C4"].value
date_end = sheet_params['C5'].value
area_rentable_office = sheet_params["C6"].value
#report_sum = sheet_params["C7"].value

# generate reporting daterange based on start and end dates
my_reporting_date_range = pd.date_range(start=date_start, end=date_end, freq='MS')


In [3]:

# Function: Read a sheet into dataframe
def read_worksheet_into_dataframe(sheet_data):
    data = sheet_data.values
    cols = next(data)[1:]
    data = list(data)
    #idx = [r[0] for r in data]
    data = (islice(r, 1, None) for r in data)
    df_data = pd.DataFrame(data, columns=cols)

    # drop rows where column Tenant is Null/NA
    df_data = df_data[df_data['Tenant'].notna()]
    return df_data


In [4]:

# Function: Calculate revenue
def calculate_revenue(date_start, date_end, df_data, product_type, report_sum):
    # Create report dataframe using datetime index for period
    df_report_rental_charge = pd.DataFrame(my_reporting_date_range, columns=['date'])   # Rental revenue
    df_report_sc_charge = pd.DataFrame(my_reporting_date_range, columns=['date'])       # Service charge revenue
    df_report_sc_charge_rate = pd.DataFrame(my_reporting_date_range, columns=['date'])       # Service charge rate
    df_report_occupancy = pd.DataFrame(my_reporting_date_range, columns=['date'])       # Occupancy rate

    # Set 'date' as the index
    df_report_rental_charge = df_report_rental_charge.set_index('date')
    df_report_sc_charge = df_report_sc_charge.set_index('date')
    df_report_occupancy = df_report_occupancy.set_index('date')
    df_pid_tenant_name_mapping = pd.DataFrame()


    # Calculate how much is SC rate based on months and years
    # April 2022: 75000
    # April 2024: 80000
    # April 2026: 85000

    sc_data = [
        [datetime(2018, 4, 1), 65000.00],
        [datetime(2020, 4, 1), 70000.00],
        [datetime(2022, 4, 1), 70000.00],
        [datetime(2024, 4, 1), 75000.00],
        [datetime(2027, 4, 1), 80000.00],
        [datetime(2030, 4, 1), 85000.00],
        [datetime(2033, 4, 1), 90000.00],
    ]

    # select rows according to product_type
    options = [product_type]
    df_by_product = df_data[df_data['Product_Type'].isin(options)]

    substring = '-'
    for index, row in df_by_product.iterrows():
        vacant = False
    
        data_area = row['Area']
        data_rental_rate = row['Rental_Rate']
        data_sc_rate = row['SC_Rate']
        data_tenant_name = row['Tenant']

        if (data_area == None) or (substring in str(data_area)):
            data_area = 0.0

        if (data_rental_rate == None) or (substring in str(data_rental_rate)):
            data_rental_rate = 0.0

        if (data_sc_rate == None) or (substring in str(data_sc_rate)):
            data_sc_rate = 0.0

        # calculate monthly rental and service charge
        calc_rental_charge = data_area * data_rental_rate
        calc_service_charge = data_area * data_sc_rate

        start = row["Start"]
        end = row["End"]

        if (pd.isna(end)) or (end == '-'):
            vacant = True
            end = date_end

        # generate name for the column
        str_level = str(row['Floor']).split('.')[0]
        str_zone = str(row['Zone'])
        if (str_level == 'None'):
            str_level = 'NA'
        if (str_zone == 'None'):
            str_zone = 'NA'
        str_sep = '-'
        str_temps = [str(str_level), str(str_zone), str(index)]
        str_column_name = str_sep.join(str_temps)
        dict01 = {'pid':[str_column_name], 'cust_name':[str_level + '-' + str_zone + '   ' + data_tenant_name]}
        df_temp = pd.DataFrame(dict01)
        df_pid_tenant_name_mapping = pd.concat([df_pid_tenant_name_mapping, df_temp], ignore_index=True)

        if (pd.isna(start)) or (start == '-'):
            start = date_start
            
        tenant_date_range = pd.date_range(start=start, end=end, freq='MS')




        if not vacant:

            tenant_date_range = pd.date_range(start=start, end=end, freq='MS')

            # generate rental charge
            df_tenant_rental_charge = pd.DataFrame(tenant_date_range, columns=['date']) # create new df with only 1 column called 'date' and fill with the date range from lcd to led
            df_tenant_rental_charge = df_tenant_rental_charge.set_index('date') # set 'date' as the index
            df_tenant_rental_charge[str_column_name] = calc_rental_charge # add a new column called <num> and fill all with the rental_charge value
            df_report_rental_charge = df_report_rental_charge.join(df_tenant_rental_charge, how="left")

            # generate service charge report
            df_sc_data = pd.DataFrame(data=sc_data, columns=['date', 'sc']).set_index('date')
            sc_date_range = pd.date_range(start=df_sc_data.index.values.min(), end=df_sc_data.index.values.max(), freq='D')
            df_sc = pd.DataFrame(sc_date_range, columns=['date'])
            df_sc = df_sc.set_index('date')

            for mydate in df_sc_data.index:
                date1 = mydate
                date2 = mydate+relativedelta(years=2)-relativedelta(days=1)
            #print('{0} - {1}'.format(date1.strftime('%Y-%m-%d'), date2.strftime('%Y-%m-%d')))
                if data_sc_rate == 0:
                    df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = 0
                elif data_sc_rate == 84100:
                    df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = 84100
                else:
                    df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = df_sc_data.loc[mydate, 'sc']

            df_sc_rate_temp = df_sc
            df_sc = df_sc * data_area

            df_tenant_service_charge = pd.DataFrame(tenant_date_range, columns=['date'])
            df_tenant_service_charge = df_tenant_service_charge.set_index('date')
            df_tenant_service_charge = df_tenant_service_charge.join(df_sc, how='left')
            df_report_sc_charge = df_report_sc_charge.join(df_tenant_service_charge, how="left")

            df_tenant_service_charge_rate = pd.DataFrame(tenant_date_range, columns=['date'])
            df_tenant_service_charge_rate = df_tenant_service_charge_rate.set_index('date')
            df_tenant_service_charge_rate = df_tenant_service_charge_rate.join(df_sc_rate_temp, how='left')
            df_report_sc_charge_rate = df_report_sc_charge_rate.join(df_tenant_service_charge_rate, how="left")
            # generate occupancy report
            df_tenant_occupancy = pd.DataFrame(tenant_date_range, columns=['date'])
            df_tenant_occupancy = df_tenant_occupancy.set_index('date')
            df_tenant_occupancy[str_column_name] = data_area
            df_report_occupancy = df_report_occupancy.join(df_tenant_occupancy, how="left")

        
        else: # if vacant
            # generate occupancy report
            df_tenant_occupancy = pd.DataFrame(tenant_date_range, columns=['date'])
            df_tenant_occupancy = df_tenant_occupancy.set_index('date')
            df_tenant_occupancy[str_column_name] = -1.0
            df_report_occupancy = df_report_occupancy.join(df_tenant_occupancy, how="left")


    # Clean dataframes
    df_report_rental_charge.fillna(0, inplace=True)
    df_report_sc_charge.fillna(0, inplace=True)
    df_report_occupancy.fillna(0, inplace=True)

    # sum each rows
    df_report_rental_charge['sum'] = df_report_rental_charge.sum(axis=1)
    df_report_sc_charge['sum'] = df_report_sc_charge.sum(axis=1)
    df_report_occupancy['sum'] = df_report_occupancy.sum(axis=1)
    
    # create summary report
    df_sum = pd.DataFrame()
    df_sum['Rental'] = df_report_rental_charge.resample(report_sum).sum()['sum']
    df_sum['SC'] = df_report_sc_charge.resample(report_sum).sum()['sum']
    df_sum['Total'] = df_sum.sum(axis=1)

    df_sum['Occ'] = df_report_occupancy.resample(report_sum).mean()['sum']
    df_sum['OccPct'] = df_sum['Occ']/area_rentable_office
    df_sum.reset_index(inplace=True)
    return df_sum, df_report_rental_charge, df_report_sc_charge, df_report_occupancy, df_report_sc_charge_rate, df_pid_tenant_name_mapping


df_data = read_worksheet_into_dataframe(sheet_data)
df_sum, df_report_rental_charge, df_report_sc_charge, df_report_occupancy, df_report_sc_charge_rate, df_pid_tenant_name_mapping = calculate_revenue(date_start, date_end , df_data, 'Office', 'Y')


## This is how to remove NaT

In [5]:
df_data.head(5).replace({np.nan: None},)

,Location,Floor,Zone,Product_Type,Group,Tenant,Area,Chg_Type,Start,End,Rental_Rate,SC_Rate,Notes_1
0,TE,PH,All,Office,Y,Medco Energi International,1413.90,A,2016-01-01 00:00:00,2034-06-30 00:00:00,0,84100,None
1,TE,55,All,Office,Y,Medco Energi International,1326.08,A,2016-01-01 00:00:00,2034-06-30 00:00:00,347395.833333,84100,None
2,TE,53,All,Office,Y,Medco Energi International,1625.42,A,2016-01-01 00:00:00,2034-06-30 00:00:00,347395.833333,84100,None
3,TE,52,All,Office,Y,MEPN,1629.22,A,2019-07-01 00:00:00,2034-06-30 00:00:00,347395.833333,84100,None
4,TE,51,C,Office,Y,MEI,396.41,A,None,2034-06-30 00:00:00,347395.833333,84100,None


## Format Style for dataframe

In [6]:
def format_time_nat(t, fmt='{:%Y.%m}'):
    try:
        return fmt.format(t)
    except ValueError:
        return t

df_data.head(5).style.format(
    {
        "Area": "{:20,.2f}",
        "Rental_Rate": "{:20,.2f}",
        "SC_Rate": "{:20,.0f}",
        "Start": format_time_nat,
        "End": format_time_nat,
        },
    # {}
)

,Location,Floor,Zone,Product_Type,Group,Tenant,Area,Chg_Type,Start,End,Rental_Rate,SC_Rate,Notes_1
0,TE,PH,All,Office,Y,Medco Energi International,"1,413.90",A,2016.01,2034.06,0.00,"84,100",None
1,TE,55,All,Office,Y,Medco Energi International,"1,326.08",A,2016.01,2034.06,"347,395.83","84,100",None
2,TE,53,All,Office,Y,Medco Energi International,"1,625.42",A,2016.01,2034.06,"347,395.83","84,100",None
3,TE,52,All,Office,Y,MEPN,"1,629.22",A,2019.07,2034.06,"347,395.83","84,100",None
4,TE,51,C,Office,Y,MEI,396.41,A,NaT,2034.06,"347,395.83","84,100",None


# STREAMLIT

In [7]:
st.set_page_config(layout = "wide")

## B/W Heatmap

In [8]:
# B/W Heatmap of occupancy
df_occ_stat = df_report_occupancy.drop(['sum'], axis = 1)
new_col_names = df_pid_tenant_name_mapping['cust_name'].tolist()
# new_col_names = ['date'] + new_col_names
df_occ_stat.columns = new_col_names

df_occ_stat.mask(df_occ_stat > 1, 1, inplace =True) # change to 0 and 1 (1 is for values greater than 0)


In [9]:
df_occ_stat[df_occ_stat.columns[::-1]] # reverse column order
df_occupied = df_occ_stat.filter(regex="[a-zA-Z0-9]$", axis=1)
df_occupied.iloc[:,30:35]

,26-D {Vacant} -> JOB Pertamina Simenggaris,25-A MEPI,25-B AON Indonesia,25-D Macmahon Indonesia,"25-E,F Medco Mining"
date,,,,,
2022-01-01,0.0,1.0,1.0,1.0,1.0
2022-02-01,0.0,1.0,1.0,1.0,1.0
2022-03-01,0.0,1.0,1.0,1.0,1.0
2022-04-01,0.0,1.0,1.0,1.0,1.0
2022-05-01,0.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...
2032-08-01,1.0,1.0,0.0,0.0,1.0
2032-09-01,1.0,1.0,0.0,0.0,1.0
2032-10-01,1.0,1.0,0.0,0.0,1.0


In [10]:
fig4 = px.imshow(df_occupied[df_occupied.columns[::-1]].loc['2022':'2032'], color_continuous_scale="gray", height=1000)
fig4.update_traces(xgap = 1, ygap = 1)
fig4.show()


In [11]:
df_vacant = df_occ_stat.filter(regex="\{Vacant\}$", axis=1)
fig4 = px.imshow(df_vacant[df_vacant.columns[::-1]].loc['2022':'2032'], color_continuous_scale="gray", height=1000)
fig4.update_traces(xgap = 1, ygap = 1)
fig4.show()


In [12]:

# df_sum.to_excel('test1.xlsx')


fig = px.bar(
    df_sum,
    x = 'date',
    y = 'OccPct',
    title = "Occupancy Rates",
)
fig.update_layout(xaxis_tickangle = -45)
fig.show()


In [13]:

fig = px.bar(
    df_sum,
    x = 'date',
    y = ["Rental", "SC"],
    title = "Revenue",
    labels = {'Rental':'Rp'},
)
fig.update_layout(xaxis_tickangle = -45)
fig.show()


In [14]:

format_mapping = {
    "Rental": "Rp {:,.2f}",
    "SC": "Rp {:,.2f}",
    "Total": "Rp {:,.2f}",
    "Occ": "{:,.2f} m2",
    "OccPct": "{:,.2%}",
}

# print dataframe
df_sum_styled = df_sum.style.format(format_mapping)
df_sum_styled


,date,Rental,SC,Total,Occ,OccPct
0,2022-12-31 00:00:00,"Rp 205,309,239,968.80","Rp 43,246,250,751.93","Rp 248,555,490,720.73","48,909.43 m2",80.53%
1,2023-12-31 00:00:00,"Rp 210,420,935,458.59","Rp 45,183,702,308.52","Rp 255,604,637,767.11","50,739.11 m2",83.54%
2,2024-12-31 00:00:00,"Rp 221,916,332,903.52","Rp 49,120,132,106.11","Rp 271,036,465,009.63","53,637.41 m2",88.31%
3,2025-12-31 00:00:00,"Rp 221,176,481,983.52","Rp 49,394,016,289.32","Rp 270,570,498,272.85","53,525.80 m2",88.13%
4,2026-12-31 00:00:00,"Rp 219,709,322,543.52","Rp 12,349,066,572.33","Rp 232,058,389,115.85","53,322.64 m2",87.79%
5,2027-12-31 00:00:00,"Rp 214,157,428,303.52","Rp 36,858,629,766.65","Rp 251,016,058,070.17","52,021.67 m2",85.65%
6,2028-12-31 00:00:00,"Rp 206,003,352,983.52","Rp 47,837,536,222.20","Rp 253,840,889,205.72","50,369.50 m2",82.93%
7,2029-12-31 00:00:00,"Rp 206,003,352,983.52","Rp 11,959,384,055.55","Rp 217,962,737,039.07","50,369.00 m2",82.93%
8,2030-12-31 00:00:00,"Rp 206,003,352,983.52","Rp 36,836,738,366.30","Rp 242,840,091,349.83","50,369.00 m2",82.93%
9,2031-12-31 00:00:00,"Rp 206,003,352,983.52","Rp 49,115,651,155.07","Rp 255,119,004,138.60","50,369.00 m2",82.93%


In [15]:

# Heatmap chart
df = df_report_rental_charge.loc['2023']
df = df.drop(['sum'], axis = 1)
df = df.T
fig2 = px.density_heatmap(df)
fig2.show()


In [16]:

fig3 = px.colors.qualitative.swatches()
fig3.show()


In [17]:
df.filter(regex='vacant', axis=1)

date
PH-All-0
55-All-1
53-All-2
52-All-3
51-C-4
...
3-All-103
1-All-104
1-All-105
1-All-106


In [18]:
df.head(5)

date,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01
PH-All-0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
55-All-1,4.606747e+08,4.606747e+08,4.606747e+08,4.606747e+08,4.606747e+08,4.606747e+08,4.606747e+08,4.606747e+08,4.606747e+08,4.606747e+08,4.606747e+08,4.606747e+08
53-All-2,5.646641e+08,5.646641e+08,5.646641e+08,5.646641e+08,5.646641e+08,5.646641e+08,5.646641e+08,5.646641e+08,5.646641e+08,5.646641e+08,5.646641e+08,5.646641e+08
52-All-3,5.659842e+08,5.659842e+08,5.659842e+08,5.659842e+08,5.659842e+08,5.659842e+08,5.659842e+08,5.659842e+08,5.659842e+08,5.659842e+08,5.659842e+08,5.659842e+08
51-C-4,1.377112e+08,1.377112e+08,1.377112e+08,1.377112e+08,1.377112e+08,1.377112e+08,1.377112e+08,1.377112e+08,1.377112e+08,1.377112e+08,1.377112e+08,1.377112e+08
